# Hobbies Network - Data Visualization Final Project 

##### Leonardo Crescenzi, Simone Lu, Enrico Romano

## Introduction

We took this dataset from 
[kaggle](https://www.kaggle.com/code/ankur310794/network-analysis-of-hobbies-interests/data) to conduct the analysis. 
Data are collected in 2013 from statistics students in FSEV UK who invited their friends to participate in the survey. 

The dataset (responses.csv) consists of 1010 rows and 150 columns (139 integer and 11 categorical).
All participants were of Slovakian nationality, aged between 15-30.

At the end of the analysis we grouped k clusters sharing more or less the same hobbies. Our idea is <b>to be able to sell these insights to bookstores or multinational companies for strategies to be applied in the film and music departments</b>. For instance, by knowing more about the people's hobby companies can place items closer on the shelf to increase the sale.

First of all, we imported all the necessary libraries for drawing our network. Then we load the csv file of hobbies and interest.

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns
import networkx as nx
from networkx.algorithms import community
from itertools import chain
import matplotlib.pyplot as plt
import igraph as ig
import chart_studio.plotly as py
from plotly.offline import iplot

pyo.init_notebook_mode()

In [2]:
df=pd.read_csv('responses.csv')
df.head()

Music  Slow songs or fast songs  Dance  Folk  Country  Classical music  \
0    5.0                       3.0    2.0   1.0      2.0              2.0   
1    4.0                       4.0    2.0   1.0      1.0              1.0   
2    5.0                       5.0    2.0   2.0      3.0              4.0   
3    5.0                       3.0    2.0   1.0      1.0              1.0   
4    5.0                       3.0    4.0   3.0      2.0              4.0   

   Musical  Pop  Rock  Metal or Hardrock  ...   Age  Height  Weight  \
0      1.0  5.0   5.0                1.0  ...  20.0   163.0    48.0   
1      2.0  3.0   5.0                4.0  ...  19.0   163.0    58.0   
2      5.0  3.0   5.0                3.0  ...  20.0   176.0    67.0   
3      1.0  2.0   2.0                1.0  ...  22.0   172.0    59.0   
4      3.0  5.0   3.0                1.0  ...  20.0   170.0    59.0   

   Number of siblings  Gender  Left - right handed                Education  \
0                 1.0  female         right handed  college/bachelor degree   
1                 2.0  female         right handed  college/bachelor degree   
2                 2.0  female         right handed         secondary school   
3                 1.0  female         right handed  college/bachelor degree   
4                 1.0  female         right handed         secondary school   

   Only child  Village - town  House - block of flats  
0          no         village          block of flats  
1          no            city          block of flats  
2          no            city          block of flats  
3         yes            city          house/bungalow  
4          no         village          house/bungalow  

[5 rows x 150 columns]

After a brief look to the dataset, we decided to select only music and movie genres, namely:

Music = ['Dance', 'Folk', 'Country', 'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock', 'Punk', 'Hiphop, Rap', 'Reggae, Ska', 'Swing, Jazz', 'Rock n roll', 'Alternative', 'Latino', 'Techno, Trance', 'Opera']

Movie = ['Horror', 'Thriller', 'Comedy', 'Romantic', 'Sci-fi', 'War', 'Fantasy/Fairy tales', 'Animated', 'Documentary', 'Western', 'Action']



In [3]:
df1=df[['Dance', 'Folk', 'Country', 'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock', 'Punk', 'Hiphop, Rap', 'Reggae, Ska', 'Swing, Jazz', 'Rock n roll', 'Alternative', 'Latino', 'Techno, Trance', 'Opera', 'Horror', 'Thriller', 'Comedy', 'Romantic', 'Sci-fi', 'War', 'Fantasy/Fairy tales', 'Animated', 'Documentary', 'Western', 'Action']]


In [4]:
df1.head()

Dance  Folk  Country  Classical music  Musical  Pop  Rock  \
0    2.0   1.0      2.0              2.0      1.0  5.0   5.0   
1    2.0   1.0      1.0              1.0      2.0  3.0   5.0   
2    2.0   2.0      3.0              4.0      5.0  3.0   5.0   
3    2.0   1.0      1.0              1.0      1.0  2.0   2.0   
4    4.0   3.0      2.0              4.0      3.0  5.0   3.0   

   Metal or Hardrock  Punk  Hiphop, Rap  ...  Thriller  Comedy  Romantic  \
0                1.0   1.0          1.0  ...       2.0     5.0       4.0   
1                4.0   4.0          1.0  ...       2.0     4.0       3.0   
2                3.0   4.0          1.0  ...       4.0     4.0       2.0   
3                1.0   4.0          2.0  ...       4.0     3.0       3.0   
4                1.0   2.0          5.0  ...       4.0     5.0       2.0   

   Sci-fi  War  Fantasy/Fairy tales  Animated  Documentary  Western  Action  
0     4.0  1.0                  5.0       5.0          3.0      1.0     2.0  
1     4.0  1.0                  3.0       5.0          4.0      1.0     4.0  
2     4.0  2.0                  5.0       5.0          2.0      2.0     1.0  
3     4.0  3.0                  1.0       2.0          5.0      1.0     2.0  
4     3.0  3.0                  4.0       4.0          3.0      1.0     4.0  

[5 rows x 28 columns]

### Features engineering

We have build a correlation matrix in order to understand the correlation between our variables. By defining the correlation, we could extract the extent to which two variables are linearly related.

Later, these correlation will be used to define the edges of our network.

In [5]:
corr = df1.corr().abs()

In [6]:
links = corr.stack().reset_index()

So, we have created a new dataset with 3 columns: 
- <b> var1 </b>: that refers to the source of the network;
- <b>var2</b>: the targets;
- <b>value</b>: basicaly, it is the correlation between the variables. It represents the weight of each connection.

In [7]:
# Transform it in a links data frame (3 columns only):
links = corr.stack().reset_index()
links.columns = ['var1', 'var2', 'value']

In [8]:
links.head(10)

var1               var2     value
0  Dance              Dance  1.000000
1  Dance               Folk  0.065654
2  Dance            Country  0.058415
3  Dance    Classical music  0.089148
4  Dance            Musical  0.074097
5  Dance                Pop  0.431962
6  Dance               Rock  0.134825
7  Dance  Metal or Hardrock  0.234597
8  Dance               Punk  0.154735
9  Dance        Hiphop, Rap  0.373528

Then, we selected the self correaltion and the cases when the correlation between two variables are *greater than 0.20*.

In [9]:
# Keep only correlation over a threshold and remove self correlation (cor(A,A)=1)
links_filtered=links.loc[ (links['value'] > 0.2) & (links['var1'] != links['var2']) ]

## Task 1: Plots 
In order to complete this task, first of all we have to build our network. So, first of all we created the edge list and consider as weight the column value, that refers to the correlation between variables. 


In [10]:
G=nx.from_pandas_edgelist(links_filtered, 'var1', 'var2')

In [11]:
g = nx.from_pandas_edgelist(links_filtered,source='var1',target='var2',edge_attr='value')

In [12]:
# print the number of nodes
print('Number of nodes is:', g.number_of_nodes())

Number of nodes is: 28


In [13]:
# print the number of edges
print('Number of edges is:',g.number_of_edges())

Number of edges is: 85


Also, we created two list of unique value of each variable in var1 and var2.

In [14]:
A = list(links_filtered['var1'].unique())

B = list(links_filtered['var2'].unique())

From these lists of variables, we obtained the node list, that basically is the sum between the nodes in set A and nodes in set B

In [15]:
node_list = set(A+B)

In [16]:
G = nx.Graph() 


Graph api to create an empty graph. And the below cells we will create nodes and edges and add them to our graph

In [17]:
for i in node_list:
    G.add_node(i)

In [18]:
for i,j in links_filtered.iterrows():
    G.add_edges_from([(j["var1"],j["var2"])])

<b>An adjacency matrix</b>

It is a way of representing a graph as a matrix of booleans (0's and 1's).
This help us to preview how is our network and which nodes are connected.

In [19]:
fig = px.imshow(nx.adjacency_matrix(G).todense(), text_auto=True, x = list(G.nodes()), y = list(G.nodes()))
fig.show()

Starting with the Network we built, we formulated questions to be answered by the analyses we will conduct with plotly library. 

<b>Questions</b>:

1. What is the graphical representation of our network?

2. How many communities are in the network? What insight can we extract from out communities?

3. What is the highest number of degrees? And how many indexes have that value?


In [20]:
edge_style_def = dict(color = 'black', width = 0.7)  # define the default style for edges
node_style_def = dict(symbol = 'circle', size = 15, opacity = 0.9, 
                      color = 'mediumseagreen', line = dict(color = 'rgb(0,0,0)', width = 0.5))

In [21]:
def set_layout(G, ndim = 2, kamada = True):    
    print('Calculating coordinates')
    layout = nx.kamada_kawai_layout(G, dim = ndim) if kamada else nx.circular_layout(G, dim = ndim) # define the node layout with NetworkX
    print('End')
    edgelist = list(G.edges())
    
    # Extract coordinates
    ax_nodes = pd.DataFrame(layout).transpose() 
    ax_nodes.columns = ['X', 'Y'] if ndim == 2 else ['X', 'Y', 'Z'] # 2D or 3D?
    
    
    # define layout for edges
    if ndim == 2:
        layout_e = dict(map(lambda e: 
                (e, # edge key
                ([layout[e[0]][0], layout[e[1]][0], None],  # X
                [layout[e[0]][1], layout[e[1]][1], None])  # Y
                ), edgelist))
    else:
        layout_e = dict(map(lambda e: 
            (e, # edge key
            ([layout[e[0]][0], layout[e[1]][0], None], # X
            [layout[e[0]][1], layout[e[1]][1], None],  # Y
            [layout[e[0]][2], layout[e[1]][2], None])  # Z
            ), edgelist))
    
    
    # extract coordinates
    ax_edges = pd.DataFrame(layout_e).transpose()
    ax_edges.columns = ['X', 'Y'] if ndim == 2 else ['X', 'Y', 'Z']

    return ax_nodes, ax_edges

In [22]:
def set_trace(X, Y, kind, style, name, Z = None, text = None): #draw nodes or edges
    mode = 'markers' if kind == 'node' else 'lines'
    hoverinfo = 'text' if kind == 'node' else 'none'
    if Z is None:
        if kind == 'node':
            return go.Scatter(x = X, y = Y, mode = mode, name = name, marker = style, text = text, hoverinfo = hoverinfo)  # draw nodes (i.e., markers) in 2D
        else:
            return go.Scatter(x = X, y = Y, mode = mode, name = name, line = style, text = text, hoverinfo = hoverinfo)  # draw edges (i.e., lines) in 2D
    else: 
        if kind == 'node':
            return go.Scatter3d(x = X, y = Y, z = Z, mode = mode, marker = style,  text = text, name = name, hoverinfo = hoverinfo)  # draw nodes (i.e., markers) in 3D
        else:
            return go.Scatter3d(x = X, y = Y, z = Z, mode = mode, line = style,  text = text, name = name, hoverinfo = hoverinfo)  # draw edges (i.e., lines) in 3D

    
def set_traces(nodes_ax: pd.DataFrame, edges_ax: pd.DataFrame, communities = None):  # Function to set up the main figure with nodes, edges, and communities
    traces = []
    Xe = list(chain(*edges_ax.X.tolist())) 
    Ye = list(chain(*edges_ax.Y.tolist()))
    Ze = list(chain(*edges_ax.Z.tolist())) if 'Z' in edges_ax.columns else None

    traces.append(set_trace(Xe, Ye, Z = Ze, kind = 'edge', style = edge_style_def, name = 'Edges'))
    
    if communities is None:
        traces.append(set_trace(nodes_ax.X.tolist(), nodes_ax.Y.tolist(), 
                                Z = nodes_ax.Z.tolist() if 'Z' in nodes_ax.columns else None, 
                                kind = 'node', style = node_style_def, 
                                name = 'Hobbies', text = nodes_ax.index))
    else:
        colors = list(sns.color_palette(n_colors =len(communities)).as_hex())
        for idcom, community in enumerate(communities):
            node_style = node_style_def.copy()
            node_style['color'] = colors[idcom]
            temp = nodes_ax[nodes_ax.index.isin(community)]
            traces.append(set_trace(temp.X.tolist(), temp.Y.tolist(), 
                                Z = temp.Z.tolist() if 'Z' in temp.columns else None, 
                                kind = 'node', style = node_style, 
                                name = 'Community '+str(idcom), text = temp.index))
            
                                       
    return traces

In [23]:
def draw_plotly_network(net, ndim = 2, communities = None, kamada = True):  # Main function to draw a network
    nodes_ax, edges_ax = set_layout(net, ndim = ndim, kamada = kamada)  # the kamada parameter sets the Networkx layout to use. The ndim parameter sets the dimensions the layout (i.e., 2D or 3D)
    traces = set_traces(nodes_ax, edges_ax, communities)  # the kamada parameter set the Networkx layout to use.
    pyo.iplot(traces, filename = 'basic-line')  # Plot the the network
    
    

#### Question 1: What is the graphical representation of our network?
We have done three different plots: a 3D plot and two 2D plot. 
We compared the Kamada layout and the Spring layout.

In [35]:
draw_plotly_network(G, kamada = True)

Calculating coordinates
End


In [25]:
draw_plotly_network(G, ndim = 3,kamada=True) 

Calculating coordinates
End


In [26]:
draw_plotly_network(G, kamada = False) 

Calculating coordinates
End


We have done three different plots in order to compare different layouts:
- First plot is done with the kamada layout;
- Second plot is done with the 3D layout, in order to make the graph more interactive;
- Third plot is done with the spring layout.

From the plots above, we can give a first look to the interactions inside our network. For example, we can notice that if someone  is interested in Fantasy, probably is interested also in Comedy, Folk and Romantic. More, if someone is interested in Documentary, he probably likes also War, Western films and Country music.

Moreover,  we can notice that all the nodes have at least one connections. 

#### Question 2
-  How many communities are in the network? According to communities what kind of observations we can do?

From the graph below, we can see that our network is composed by four different communities. 

In order to detect the number of communities, we have used the greedy modularity algorithm and the Louvain algorithm.


In [27]:
comm_mod = community.greedy_modularity_communities(G)

In [28]:
G_hobbies = nx.Graph(G)
# Find the communities
communities = sorted(community.greedy_modularity_communities(G), key=len, reverse=True)
# Count the communities
print(f"The Hobbies dataset has {len(communities)} communities.")

The Hobbies dataset has 4 communities.


## Greedy Modularity

This function uses Clauset-Newman-Moore greedy modularity maximization [2] to find the community partition with the largest modularity.

Greedy modularity maximization begins with each node in its own community and repeatedly joins the pair of communities that lead to the largest modularity until no futher increase in modularity is possible (a maximum). (Networx, n.d)

In [29]:
draw_plotly_network(G, communities = comm_mod, kamada=True)

Calculating coordinates
End


## Louvain Community

Find the best partition of a graph using the Louvain Community Detection Algorithm.
Louvain Community Detection Algorithm is a simple method to extract the community structure of a network. This is a heuristic method based on modularity optimization. 
It maximizes a modularity score for each community, where the modularity quantifies the quality of an assignment of nodes to communities. This means that the algorithm evaluates how much more densely connected the nodes within a community are, compared to how connected they would be in a random network.
 

In [30]:
comm_lou = community.louvain_communities(G)  # Detect communities using a Louvain algorithm
draw_plotly_network(G, communities = comm_lou)   # Draw the network using the Kamada Kawai 2D layout

AttributeError: module 'networkx.algorithms.community' has no attribute 'louvain_communities'

In [ ]:
def get_node2community(communities): # Communities to DataFrame
    node2community = []
    #Loop through each community in the network
    for community_number, community in enumerate(communities):
        node2community += list(zip(community, [community_number]* len(community)))
    return pd.DataFrame(node2community, columns = ['Node', 'Class'])

In [ ]:
get_node2community(comm_mod)

From the community detection, we obtain 4 cluster. Each cluster represent people with similar interests. For instace, if a person likes 'Horror' films, he will be in cluster 2 and probably he has also interest in Documentary and War films. Instead, if a person likes Pop music, he will be in cluster 3 and probably likes also Hiphop and Dance. 

#### Question 3
- What is the highest number of degrees? And how many indexes have that value?


In order to answer this question, we have defined a plotly function to obtain histogram degree and the sequence of degrees rank. 

In [ ]:
degrees=[]

for v in G.nodes():
    degrees.append(G.degree(v))


print(degrees)

In [ ]:
degrees = [G.degree(n) for n in G.nodes()]
print(degrees)

In [ ]:
def degree_rank(net):
    degree_sequence = sorted((d for n, d in net.degree()), reverse=True)
    df = pd.DataFrame(degree_sequence, columns = ['Degree'])
    fig = px.scatter(df, y = 'Degree')
    fig.update_layout(xaxis_title="Rank")
    return fig

def degree_hist(net, kind = None, group = False):
    if kind == 'in':  degrees = net.in_degree()
    elif kind == 'out':  degrees = net.out_degree()
    else: degrees = net.degree()
   
    df = pd.DataFrame(degrees, columns = ['Node', 'Degree'])
    nbins = None if group else len(degrees)
    fig = px.histogram(df, x="Degree", nbins = nbins)
    fig.update_layout(yaxis_title="# of Nodes", bargap=0.01)
    return fig  

In [ ]:
degree_hist(G)

In [ ]:
degree_rank(G)

Thanks to the degree histogram, we can notice that the most common degree is 5. Moreover, with the degree rank plot, we can also notice that five indexes have five degrees. 

After answering these three questions with plots, we can start the network analysis.

## Task 2. Network Analysis

Network measures:
- degree centrality
- betweenness
- clustering coefficient of a node

To better understanding our network we started by visualizing the network and then we passed to answering the questions above:

In [ ]:
# lets visualize the graph
plt.figure(figsize=(10,7))
nx.draw(G,with_labels=True)

- is the graph connected?

- is the graph bipartite?

- is the graph scale-free?

- diameter

- average degree

In [ ]:
# is the graph connected?
connections=nx.is_connected(G)
# is the graph bipartite?
bipartite=nx.is_bipartite(G)
# Diameter
diameter=nx.diameter(G)
#is the graph directed?
directed=nx.is_directed(G)
# avg degree
avg_degree=sum(dict(G.degree()).values()) / nx.number_of_nodes(G)

print('Is the graph connected?',connections)
print('Is the graph bipartite?', bipartite)
print('What is the diameter of the network?',diameter)
print('Is the graph directed?',directed)
print('What is the average degree of the nodes?',avg_degree)

In [ ]:
nx.average_clustering(G)

By density we mean how many edges the graph has compared to the nodes. 
the density of a complete graph is 1.

In [ ]:
nx.density(G)

## Node Importance -  Degree 

Node importance represents how connected a node is to other networks. A node that is connected to many other nodes is generally considered an important node. 

To better understand the relationships between the variables we studied the degree of centrality of our network. What we found out was  that "Musical" and "Swing, Jazz" are the most connected nodes of our networks, "Horror", instead, has the lowest degree. By visualizing the network belove, is easier to watch the number of the connections thanks to the scale of orange, from lighter, few connections, to darker, many connections.

In the following code we printed out the nodes with highest degrees.

In [ ]:
# lets sort the nodes according to their degree centrality and output the first 5 nodes

sorted_degrees = sorted(G.degree, key=lambda x:x[1], reverse = True)
print(sorted_degrees[0:5])

The degree centrality of a node is simply its degree—the number of edges it has. The higher the degree, the more central the node is. This can be an effective measure, since many nodes with high degrees also have high centrality by other measures.

In [ ]:
deg_cen = nx.degree_centrality(G)
deg_df = pd.DataFrame(deg_cen.items(), columns = ['Node', 'Degree'],)
deg_df = deg_df.sort_values(by=['Degree'], ascending=False)
deg_df.sort_values('Degree',ascending=False).head(10)

In [ ]:
np.random.seed(123)

plt.figure(figsize=(15,8))

node_color2 = np.fromiter(deg_cen.values(), float)

nx.draw_networkx(G, 
                 node_color=node_color2, alpha=0.7, 
                 with_labels=True, edge_color='0.4', cmap='Oranges')
plt.savefig('degree.png')

## Node Importance -  Betweenness
Betweenness centrality of a node v is the sum of the fraction of all-pairs shortest paths that pass through v:


The largest diameter between two nodes is 6. The farther nodes are "Horror" and "Animated", as shown by the Betweenness value equal to 0 and from the graph below thanks to the scale of blue.  

Betweenness centrality of a node v is the sum of the fraction of all-pairs shortest paths that pass through v

In [ ]:
bw_centrality_g = nx.betweenness_centrality(G, normalized=False)
print(bw_centrality_g)

In [ ]:
# Find the largest connected components
# Connected components are sorted in descending order of their size
largest_cc_nodeset = max(nx.connected_components(G), key=len)
largest_cc = G.subgraph(largest_cc_nodeset).copy()
print(largest_cc.nodes())

In [ ]:
print(nx.diameter(largest_cc))

In [ ]:
path_l = dict(nx.shortest_path_length(G))
path_tuple = [] 
for k, v in path_l.items():
    for h, vv in v.items(): 
        path_tuple.append((k, h, vv))
path_df = pd.DataFrame(path_tuple, columns = ['NodeA', 'NodeB', 'SH Length']) 
path_df

In [ ]:
# Betweenness centrality
centr = nx.betweenness_centrality(largest_cc)
centr_df = pd.DataFrame(centr.items(), columns = ['Node', 'Betweeness'])
centr_df=centr_df.sort_values('Betweeness',ascending=False)

In [ ]:
np.random.seed(123)

plt.figure(figsize=(15,8))

node_color3 = np.fromiter(centr.values(), float)

nx.draw_networkx(G, 
                 node_color=node_color3, alpha=0.7, 
                 with_labels=True, edge_color='0.4', cmap='Blues')
plt.savefig('between1.png')

## Clustering coefficient of node
It is defined as the likelihood that any two nodes with a common neighbour are themselves connected. Also here we can see as "Horror" is very different compared to other nodes. We also printed three top ten about "Betwenness centrality", "Clustering coefficent" and "Degree centrality".  

In [ ]:
# Clustering coefficient of all nodes (in a dictionary)
cluster=nx.clustering(G)
cluster_df = pd.DataFrame(cluster.items(), columns = ['Node', 'Coefficient'])
cluster_df.sort_values('Coefficient',ascending=False).head(10)

In [ ]:
np.random.seed(123)

plt.figure(figsize=(15,8))

node_color4 = np.fromiter(cluster.values(), float)

nx.draw_networkx(G, 
                 node_color=node_color4, alpha=0.7, 
                 with_labels=True, edge_color='0.4', cmap='Greens')
plt.savefig('cluster.png')

In [ ]:
def get_top_keys(df, column, top):
    return df.sort_values(column, ascending = False).head(top)['Node'].tolist()

In [ ]:
print('Top 10 by Betweeness centrality', get_top_keys(centr_df, 'Betweeness', 10))
print('\n')
print('Top 10 by Clustering coefficient ', get_top_keys(cluster_df, 'Coefficient', 10))
print('\n')
print('Top 10 by Degree centrality', get_top_keys(deg_df, 'Degree', 10))

In [ ]:
hobbies = links_filtered.drop(axis =1 , columns = 'value')

In [ ]:
# gephi_df=links_filtered.rename(columns={'var1':'Source','var2':'target'})

In [ ]:
# gephi_df.to_excel('hobbies network.xlsx')

# References 

- https://www.kaggle.com/code/ankur310794/network-analysis-of-hobbies-interests/data

- Data Visualization lab lessons.